In [6]:
#exsstract zip at data_csv/matches.zip 
import zipfile
import os
import sys
import shutil

def extract_zip(zip_file, dest_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(dest_dir)
    print("Extracted zip file: " + zip_file + " to " + dest_dir)

matches_fp = "data_csv/matches.csv"

if os.path.exists(matches_fp):
    print("matches.csv already exists")
else:
    matches_zip_fp = "data_csv/matches.zip"
    if not os.path.exists(matches_zip_fp):
        print("matches.zip does not exist")
        #throw exception
        raise Exception("matches.zip does not exist")
    extract_zip(matches_zip_fp, "data_csv")

Extracted zip file: data_csv/matches.zip to data_csv


In [5]:
import pandas as pd 
matches_data = pd.read_csv(matches_fp)

FileNotFoundError: [Errno 2] No such file or directory: 'data_csv/matches.csv'

In [7]:
champion_info = pd.DataFrame(columns=["role", "lane", "champion", "spell1", "spell2"])
role = []
lane = []
champion = []
spell1 = []
spell2 = []

for i in range(1,11):
    role.extend(matches_data["p{}_role".format(i)])
    lane.extend(matches_data["p{}_lane".format(i)])
    champion.extend(matches_data["p{}_championName".format(i)])
    spell1.extend(matches_data["p{}_summoner1Id".format(i)])
    spell2.extend(matches_data["p{}_summoner2Id".format(i)])

champion_info["role"] = role
champion_info["lane"] = lane
champion_info["champion"] = champion
champion_info["spell1"] = spell1
champion_info["spell2"] = spell2


In [ ]:
[
  {
    "id": "barrier",
    "key": "21",
  },
  {
    "id": "boost",
    "name": "Cleanse",
    "key": "1",
  },
  {
    "id": "dot",
    "name": "Ignite",
    "key": "14",
  },
  {
    "id": "exhaust",
    "name": "Exhaust",
    "key": "3",
  },
  {
    "id": "flash",
    "name": "Flash",
    "key": "4",
  },
  {
    "id": "haste",
    "name": "Ghost",
    "key": "6",
  },
  {
    "id": "heal",
    "name": "Heal",
    "key": "7",
  },
  {
    "id": "mana",
    "name": "Clarity",
    "key": "13",
  },
  {
    "id": "pororecall",
    "name": "To the King!",
    "key": "30",
  },
  {
    "id": "smite",
    "name": "Smite",
    "key": "11",
  },
  {
    "id": "snowball",
    "key": "32",
  },
  {
    "id": "teleport",
    "key": "12",
  }
]

In [8]:
champion_info

,role,lane,champion,spell1,spell2
0,NONE,JUNGLE,KogMaw,4,6
1,DUO,TOP,Fiora,4,12
2,NONE,JUNGLE,Aatrox,4,12
3,NONE,JUNGLE,Garen,14,4
4,NONE,JUNGLE,Garen,14,4
...,...,...,...,...,...
242165,SUPPORT,BOTTOM,Lulu,4,7
242166,SOLO,BOTTOM,Soraka,4,3
242167,SUPPORT,BOTTOM,Braum,3,4
242168,SUPPORT,BOTTOM,Brand,14,4


there is large portion of matches where players switched their lanes in lobby


In [18]:
jungle = champion_info[champion_info["lane"] == "JUNGLE"]
print(jungle)
jungle_without_smite = jungle[jungle.apply(lambda x: x["spell1"] != 11 and x["spell2"] != 11, axis=1)]
print(jungle_without_smite)
print("junglers without smite are percentage: ", len(jungle_without_smite)/len(jungle))

        role    lane   champion  spell1  spell2
0       NONE  JUNGLE     KogMaw       4       6
2       NONE  JUNGLE     Aatrox       4      12
3       NONE  JUNGLE      Garen      14       4
4       NONE  JUNGLE      Garen      14       4
5       NONE  JUNGLE      Garen      14       4
...      ...     ...        ...     ...     ...
241858  NONE  JUNGLE       Pyke      14       4
241989  NONE  JUNGLE       Rell       4      14
242005  NONE  JUNGLE  Seraphine       4       3
242023  NONE  JUNGLE      Janna       4      14
242153  NONE  JUNGLE       Nami       4       3

[52667 rows x 5 columns]
        role    lane   champion  spell1  spell2
0       NONE  JUNGLE     KogMaw       4       6
2       NONE  JUNGLE     Aatrox       4      12
3       NONE  JUNGLE      Garen      14       4
4       NONE  JUNGLE      Garen      14       4
5       NONE  JUNGLE      Garen      14       4
...      ...     ...        ...     ...     ...
241858  NONE  JUNGLE       Pyke      14       4
241989  NONE  

determining who plays on which lane must be determined differently

In [19]:
data_cleaned = pd.read_csv("data_csv/timestamps_ext.csv")

In [20]:
timestamps_2min = data_cleaned.loc[data_cleaned.groupby('matchId')['timeStamp'].apply(lambda x: (x - 2 ).abs().idxmin())]

In [31]:
timestamps_2min = timestamps_2min.merge(matches_data, on="matchId")

In [32]:
print(timestamps_2min.columns)
def find_in_columns( string_to_find ):
    found = []
    for c in timestamps_2min.columns:
        if str.find(c, string_to_find) != -1:
            found.append(c)
    return found
find_in_columns("p2")

Index(['matchId', 'timeStamp', 'p1_currentGold', 'p2_currentGold',
       'p3_currentGold', 'p4_currentGold', 'p5_currentGold', 'p6_currentGold',
       'p7_currentGold', 'p8_currentGold',
       ...
       't0_objectives_inhibitor_kills', 't1_objectives_inhibitor_kills',
       't0_objectives_riftHerald_first', 't1_objectives_riftHerald_first',
       't0_objectives_riftHerald_kills', 't1_objectives_riftHerald_kills',
       't0_objectives_tower_first', 't1_objectives_tower_first',
       't0_objectives_tower_kills', 't1_objectives_tower_kills'],
      dtype='object', length=1731)


['p2_currentGold',
 'p2_goldPerSecond',
 'p2_jungleMinionsKilled',
 'p2_level',
 'p2_minionsKilled',
 'p2_participantId_x',
 'p2_timeEnemySpentControlled',
 'p2_totalGold',
 'p2_xp',
 'p2_championStats_abilityHaste',
 'p2_championStats_abilityPower',
 'p2_championStats_armor',
 'p2_championStats_armorPen',
 'p2_championStats_armorPenPercent',
 'p2_championStats_attackDamage',
 'p2_championStats_attackSpeed',
 'p2_championStats_bonusArmorPenPercent',
 'p2_championStats_bonusMagicPenPercent',
 'p2_championStats_ccReduction',
 'p2_championStats_cooldownReduction',
 'p2_championStats_health',
 'p2_championStats_healthMax',
 'p2_championStats_healthRegen',
 'p2_championStats_lifesteal',
 'p2_championStats_magicPen',
 'p2_championStats_magicPenPercent',
 'p2_championStats_magicResist',
 'p2_championStats_movementSpeed',
 'p2_championStats_omnivamp',
 'p2_championStats_physicalVamp',
 'p2_championStats_power',
 'p2_championStats_powerMax',
 'p2_championStats_powerRegen',
 'p2_championStats_sp

In [36]:
timestamps_2min["p3_individualPosition"]

0        MIDDLE
1        MIDDLE
2        MIDDLE
3        MIDDLE
4        MIDDLE
          ...  
24212    MIDDLE
24213    MIDDLE
24214    MIDDLE
24215    JUNGLE
24216    MIDDLE
Name: p3_individualPosition, Length: 24217, dtype: object

In [3]:
# matches_data.head()

,matchId,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,...,t0_objectives_inhibitor_kills,t1_objectives_inhibitor_kills,t0_objectives_riftHerald_first,t1_objectives_riftHerald_first,t0_objectives_riftHerald_kills,t1_objectives_riftHerald_kills,t0_objectives_tower_first,t1_objectives_tower_first,t0_objectives_tower_kills,t1_objectives_tower_kills
0,EUW1_6883129810,1712084570608,2142,1712086738289,6883129810,CLASSIC,teambuilder-match-6883129810,1712084595523,MATCHED_GAME,14.6.570.6276,...,4,0,False,False,0,0,True,False,11,3
1,EUW1_6882416210,1712046205448,1623,1712047855825,6882416210,CLASSIC,teambuilder-match-6882416210,1712046232712,MATCHED_GAME,14.6.570.6276,...,0,2,False,True,0,1,False,True,5,10
2,EUW1_6881092720,1711972830768,1701,1711974582700,6881092720,CLASSIC,teambuilder-match-6881092720,1711972881130,MATCHED_GAME,14.6.570.6276,...,0,0,False,True,0,1,False,True,3,8
3,EUW1_6879405717,1711872829956,1525,1711874389709,6879405717,CLASSIC,teambuilder-match-6879405717,1711872863863,MATCHED_GAME,14.6.570.6276,...,0,1,True,False,1,0,True,False,3,8
4,EUW1_6879389461,1711869886553,1783,1711871742154,6879389461,CLASSIC,teambuilder-match-6879389461,1711869958422,MATCHED_GAME,14.6.570.6276,...,1,0,True,False,1,0,True,False,11,4


In [19]:
def find_in_columns( string_to_find ):
    found = []
    for c in matches_data.columns:
        if str.find(c, string_to_find) != -1:
            found.append(c)
    return found

# #check for smite as summoners spell -> jungle role

# print(matches_data[find_in_columns("role")].head())

# #https://github.com/ngryman/lol-spells/blob/master/spells.json 4 is flash; key: is summonerId
# print(matches_data[find_in_columns("summoner1Id")].head())
# print(matches_data[find_in_columns("summoner2Id")].head())
# matches_data.apply(lambda x: x[find_in_columns("summoner1Id")].value_counts(), axis=1)



  p0_role p1_role p2_role p3_role  p4_role  p5_role p6_role p7_role p8_role  p9_role
0    NONE    NONE    SOLO   CARRY  SUPPORT     SOLO    NONE    SOLO   CARRY  SUPPORT
1     DUO    NONE     DUO   CARRY  SUPPORT  SUPPORT    NONE   CARRY   CARRY  SUPPORT
2    NONE    NONE   CARRY    SOLO  SUPPORT     SOLO    NONE    SOLO   CARRY  SUPPORT
3    NONE    NONE    SOLO   CARRY  SUPPORT     SOLO    NONE    SOLO   CARRY  SUPPORT
4    NONE    NONE    SOLO   CARRY  SUPPORT     SOLO    NONE    SOLO   CARRY  SUPPORT
   p0_summoner1Id  p1_summoner1Id  p2_summoner1Id  p3_summoner1Id  p4_summoner1Id  p5_summoner1Id  p6_summoner1Id  p7_summoner1Id  p8_summoner1Id  p9_summoner1Id
0               4               4               4               6               3               4               4               4               6               4
1               4               4               4               6               4              14              11               4               6               4
2   